In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
import random
my_header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}

In [22]:
products = pd.read_csv('output/df_total.csv')

In [23]:
products.head()

,group,sub-group,product_line,product_number,description,url
0,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47459,Size:X-Small,https://products.halyardhealth.com/infection-p...
1,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47460,Size:Small,https://products.halyardhealth.com/infection-p...
2,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47461,Size:Medium,https://products.halyardhealth.com/infection-p...
3,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47462,Size:Large,https://products.halyardhealth.com/infection-p...
4,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47463,Size:X-Large,https://products.halyardhealth.com/infection-p...


In [25]:
products['mfg'] = 'Halyard Health'

In [27]:
upcs = products[['mfg', 'product_number']]
upcs['search'] = upcs['mfg'] + ' ' + upcs['product_number'].astype(str)
upcs[['case_gtin', 'box_gtin']] = 'NaN'
upcs.head()

D:\Software\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Software\anaconda3\lib\site-packages\pandas\core\indexing.py:659: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[k] = np.nan
D:\Software\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

,mfg,product_number,search,case_gtin,box_gtin
0,Halyard Health,47459,Halyard Health 47459,NaN,NaN
1,Halyard Health,47460,Halyard Health 47460,NaN,NaN
2,Halyard Health,47461,Halyard Health 47461,NaN,NaN
3,Halyard Health,47462,Halyard Health 47462,NaN,NaN
4,Halyard Health,47463,Halyard Health 47463,NaN,NaN


## First attempt using BarCodeSpider.com

Thwarted by captcha... Also, some of the products were missing...

In [ ]:
# case_found = False
# box_found = False

# for i in range(len(upcs2)):
#     search = upcs2.loc[i, 'search']
#     print('Searching product: ', search, ' on UPC_Item_DB')
    
#     search = search.replace(' ','%20')
#     url = 'https://www.upcitemdb.com/upc/' + search
#     response = requests.get(url, headers=my_header, verify=False)
    
#     if(response):
#         print('\tProduct found!')
        
#         text = BeautifulSoup(response.text, 'html.parser')
#         text = text.find_all('div', {'class': 'upclist col-xs-12'})
#         text = text[0].ul.find_all('li',{'class': ''})
#         upc_list = [t.get_text()[0:12] for t in text]

#         for upc in upc_list[0:5]:
#             print('\tChecking UPC: ', upc, ' on Barcode Spider')
            
#             url = 'https://www.barcodespider.com/' + upc
#             response = requests.get(url, headers=my_header)
#             text = BeautifulSoup(response.text, 'html.parser')
#             text = text.get_text().lower()
            
#             if text.find('model') >= 0:
#                 if text.find('case') >= 0:
#                     print('\t\tCase found!')
#                     upcs2.loc[i, 'case_gtin'] = upc
#                     case_found = True
#                 else:
#                     print('\t\tBox found!')
#                     upcs2.loc[i, 'box_gtin'] = upc
#                     box_found = True
                    
#             elif (text.find('robot') >= 0) or (text.find('captcha') >= 0):
#                 print('\tRobot detected!')
                
#             else:
#                 print('\tUPC not found.')
            
#             time.sleep(random.randint(10,15))
            
#             if case_found and box_found:
#                 print('\tCase and box found! Moving on.')
#                 break
#     else:
#         print('\tNo UPCs found on UPC_Item_DB.')
        
#     case_found = False
#     box_found = False

## Second attempt using only UpcItemDB.com

In [30]:
%%time
case_list = {}
box_list = {}
start = 0

for i in range(start, len(upcs)):    # for each product
    search = upcs.loc[i, 'search']
    print('Searching product: ', search, ' on UPC_Item_DB')
    
    search = search.replace(' ','%20')
    url = 'https://www.upcitemdb.com/upc/' + search
    response = requests.get(url, headers=my_header, verify=False)
    time.sleep(0.1)
    
    if(response):
        print('\tProduct found!')
        
        text = BeautifulSoup(response.text, 'html.parser')
        text = text.find_all('div', {'class': 'upclist col-xs-12'})
        text = text[0].ul.find_all('div',{'class': 'rImage'})
        upc_list = [t.a.get_text() for t in text]

        for upc in upc_list:   # count number of shoppping options for each upc
            print('\tChecking UPC: ', upc)
            
            url = 'https://www.upcitemdb.com/upc/' + upc
            response = requests.get(url, headers=my_header, verify=False)
            time.sleep(0.01)
            text = BeautifulSoup(response.text, 'html.parser')
            
            descr = text.find('div',{'class':'col-sm-12'}).get_text().lower()
            num_options = len(text.find('div',{'class':'tabcon-compare box row'}).find_all('tr')) - 1
            if (descr.find('case') >= 0) or (descr.find('cart') >= 0):
                print('\tcase links found: ', num_options)
                case_list[upc] = num_options
            elif descr.find('glove') >= 0:
                print('\tbox links found: ', num_options)
                box_list[upc] = num_options
            else:
                print('\tnot relevant')
            
            time.sleep(random.randint(10, 20))
    else:
        print('\tNo UPCs found on UPC_Item_DB.')
    
    # use max number of options to select best UPC
    if case_list:
        upcs.loc[i, 'case_gtin'] = sorted(case_list.items(), key=lambda x: x[1], reverse=True)[0][0]
    if box_list:
        upcs.loc[i, 'box_gtin'] = sorted(box_list.items(), key=lambda x: x[1], reverse=True)[0][0]
#     print('Updated table:')
    print('\tUpdated: ', upcs.iloc[i].case_gtin)
    print('\tUpdated: ', upcs.iloc[i].box_gtin)
    
    case_list = {}
    box_list = {}
    
    time.sleep(random.randint(10, 20))
#     break

Searching product:  Halyard Health 47459  on UPC_Item_DB


D:\Software\anaconda3\lib\site-packages\urllib3\connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.upcitemdb.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


	No UPCs found on UPC_Item_DB.
	Updated:  NaN
	Updated:  NaN
Searching product:  Halyard Health 47460  on UPC_Item_DB


D:\Software\anaconda3\lib\site-packages\urllib3\connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.upcitemdb.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


	No UPCs found on UPC_Item_DB.
	Updated:  NaN
	Updated:  NaN
Searching product:  Halyard Health 47461  on UPC_Item_DB


...

	No UPCs found on UPC_Item_DB.
	Updated:  NaN
	Updated:  NaN
Searching product:  Halyard Health 69703  on UPC_Item_DB


D:\Software\anaconda3\lib\site-packages\urllib3\connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.upcitemdb.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


	No UPCs found on UPC_Item_DB.
	Updated:  NaN
	Updated:  NaN
Searching product:  Halyard Health 69704  on UPC_Item_DB


D:\Software\anaconda3\lib\site-packages\urllib3\connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.upcitemdb.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


	No UPCs found on UPC_Item_DB.
	Updated:  NaN
	Updated:  NaN
Wall time: 2h 28min 51s


In [34]:
text.get_text()

"\n\n\n\n\n\nUPC 713145685826 - Halyard Health KIM 69455 Impervious Knit Cuffs Open Back Gown - Blue Large - 60  | upcitemdb.com \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nToggle navigation\nupcitemdb\n\n\n\n\n\n\n\n\n\t\t\t\t\t\t\t\t\t\t\tUPC \n\nUPCProduct NameBook  TitleValidator \n\n\n\n\n\n\n\nAPI\nDocumentation\n\n\n\n\n\n\n\nUPC 713145685826\n\n        UPC 713145685826 is associated with Halyard Health KIM 69455 Impervious Knit Cuffs Open Back Gown - Blue Large - 60   \n\n\n\nBusiness & Industrial > Medical > Hospital Gowns\n\n\n \n\n\n\n\n\n\n\n ZOOM\n\n  UPC 713145685826 has following Product Name Variations:\n\nHalyard Health KIM 69455 Impervious Knit Cuffs Open Back Gown - Blue Large - 60    \n\n\n\n\n\n\nMore Info\n\n\n\nUPC-A:7 13145 68582 6\nEAN-13:0 713145 685826\nCountry of Registration:United States\nBrand:                    \t\t\tHalyard Health                        \nModel #:KIM 69455\nLast Scanned:2017-10-25 10:50:45\n\n\n\n\n\n \n\n\n \n\n \nShopping Info\nPro

In [49]:
upcs = products.merge(upcs[['product_number', 'case_gtin', 'box_gtin']], on='product_number')

In [21]:
products.to_csv('output/sterile_gloves_gtin.csv')

# Debug

In [299]:
url = 'https://www.upcitemdb.com/upc/30680651550812'
response = requests.get(url, headers=my_header, verify=False)

D:\Software\anaconda3\lib\site-packages\urllib3\connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.upcitemdb.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [304]:
text = BeautifulSoup(response.text, 'html.parser')
text.get_text().lower().find('cart')

102

In [287]:
upcs2

,mfg,product,search,case_gtin,box_gtin
0,Halyard Health,50852,Halyard Health 50852,000000000000,000000000000
1,Halyard Health,50853,Halyard Health 50853,000000000000,000000000000
2,Halyard Health,55080,Halyard Health 55080,000000000000,036000550801
3,Halyard Health,52101,Halyard Health 52101,000000000000,820170072870
4,Halyard Health,52102,Halyard Health 52102,000000000000,820170072894
5,Halyard Health,52103,Halyard Health 52103,713049530543,820170072917
6,Halyard Health,55091,Halyard Health 55091,820170073709,820170073693
7,Halyard Health,55092,Halyard Health 55092,820170073723,073598346935
8,Halyard Health,55093,Halyard Health 55093,820170073747,820170073730
9,Halyard Health,47459,Halyard Health 47459,000000000000,000000000000


In [280]:
upcs

,mfg,product,search,case_gtin,box_gtin
0,Halyard Health,55081,Halyard Health 55081,823019646171,036000550818
1,Halyard Health,55082,Halyard Health 55082,30680651550829,036000550825
2,Halyard Health,55083,Halyard Health 55083,30680651550836,816687292108
3,Halyard Health,55084,Halyard Health 55084,30680651550843,741360082212
4,Halyard Health,50852,Halyard Health 50852,000000000000,000000000000
...,...,...,...,...,...
103,Halyard Health,44792,Halyard Health 44792,000000000000,000000000000
104,Halyard Health,44793,Halyard Health 44793,000000000000,000000000000
105,Halyard Health,44794,Halyard Health 44794,000000000000,000000000000
106,Halyard Health,44795,Halyard Health 44795,000000000000,000000000000


In [288]:
upcs.iloc[4:14] = upcs2.iloc[0:10]

D:\Software\anaconda3\lib\site-packages\pandas\core\indexing.py:1717: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


In [292]:
upcs.iloc[3:20]

,mfg,product,search,case_gtin,box_gtin
3,Halyard Health,55084.0,Halyard Health 55084,30680651550843,741360082212
4,Halyard Health,52102.0,Halyard Health 52102,000000000000,820170072894
5,Halyard Health,52103.0,Halyard Health 52103,713049530543,820170072917
6,Halyard Health,55091.0,Halyard Health 55091,820170073709,820170073693
7,Halyard Health,55092.0,Halyard Health 55092,820170073723,073598346935
8,Halyard Health,55093.0,Halyard Health 55093,820170073747,820170073730
9,Halyard Health,47459.0,Halyard Health 47459,000000000000,000000000000
10,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN


In [282]:
upcs2 = upcs.iloc[4:14].reset_index(drop=True)
upcs2

,mfg,product,search,case_gtin,box_gtin
0,Halyard Health,50852,Halyard Health 50852,000000000000,000000000000
1,Halyard Health,50853,Halyard Health 50853,000000000000,000000000000
2,Halyard Health,55080,Halyard Health 55080,000000000000,000000000000
3,Halyard Health,52101,Halyard Health 52101,000000000000,000000000000
4,Halyard Health,52102,Halyard Health 52102,000000000000,000000000000
5,Halyard Health,52103,Halyard Health 52103,000000000000,000000000000
6,Halyard Health,55091,Halyard Health 55091,000000000000,000000000000
7,Halyard Health,55092,Halyard Health 55092,000000000000,000000000000
8,Halyard Health,55093,Halyard Health 55093,000000000000,000000000000
9,Halyard Health,47459,Halyard Health 47459,000000000000,000000000000


# Merge Results with New Table

In [70]:
pd.set_option('max_rows', 400)        # display at least 400 rows when max is exceeded
pd.set_option('display.width', 160)   # set max width to 80 characters

In [57]:
df_total = pd.read_csv('output/df_total.csv', dtype=str)
df_total2 = pd.read_csv('output/df_total2.csv', dtype=str)
upcs2 = pd.read_csv('output/sterile_gloves_gtin.csv', index_col=0, dtype=str)

In [50]:
upcs.head()

,group,sub-group,product_line,product_number,description,url,mfg,case_gtin,box_gtin
0,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47459,Size:X-Small,https://products.halyardhealth.com/infection-p...,Halyard Health,NaN,NaN
1,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47460,Size:Small,https://products.halyardhealth.com/infection-p...,Halyard Health,NaN,NaN
2,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47461,Size:Medium,https://products.halyardhealth.com/infection-p...,Halyard Health,NaN,NaN
3,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47462,Size:Large,https://products.halyardhealth.com/infection-p...,Halyard Health,NaN,NaN
4,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47463,Size:X-Large,https://products.halyardhealth.com/infection-p...,Halyard Health,NaN,NaN


In [95]:
upcs2.head()

,mfg,product,style,description,url,case_gtin,box_gtin
0,Halyard Health,55081,PURPLE NITRILE* Exam Glove,"Size:Small, Cleared for Use with Chemotherapy ...",https://products.halyardhealth.com/infection-p...,30680651550812,094718114470
1,Halyard Health,55082,PURPLE NITRILE* Exam Glove,"Size:Medium, Cleared for Use with Chemotherapy...",https://products.halyardhealth.com/infection-p...,30680651550829,036000055085
2,Halyard Health,55083,PURPLE NITRILE* Exam Glove,"Size:Large, Cleared for Use with Chemotherapy ...",https://products.halyardhealth.com/infection-p...,30680651550836,816687292108
3,Halyard Health,55084,PURPLE NITRILE* Exam Glove,"Size:X-Large, Cleared for Use with Chemotherap...",https://products.halyardhealth.com/infection-p...,30680651550843,741360082212
4,Halyard Health,50852,PURPLE NITRILE* Exam Glove,"Size:Medium, Cleared for Use with Chemotherapy...",https://products.halyardhealth.com/infection-p...,NaN,NaN


In [56]:
df_total.head()

,group,sub-group,product_line,product_number,description,url
0,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47459,Size:X-Small,https://products.halyardhealth.com/infection-p...
1,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47460,Size:Small,https://products.halyardhealth.com/infection-p...
2,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47461,Size:Medium,https://products.halyardhealth.com/infection-p...
3,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47462,Size:Large,https://products.halyardhealth.com/infection-p...
4,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47463,Size:X-Large,https://products.halyardhealth.com/infection-p...


In [58]:
df_total2.head()

,mfg,group,sub-group,product_line,product_number,description,url
0,Halyard Health,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47459,"Size:X-Small, Brand:STERLING ZERO*, Color:Ster...",https://products.halyardhealth.com/infection-p...
1,Halyard Health,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47460,"Size:Small, Brand:STERLING ZERO*, Color:Sterli...",https://products.halyardhealth.com/infection-p...
2,Halyard Health,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47461,"Size:Medium, Brand:STERLING ZERO*, Color:Sterl...",https://products.halyardhealth.com/infection-p...
3,Halyard Health,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47462,"Size:Large, Brand:STERLING ZERO*, Color:Sterli...",https://products.halyardhealth.com/infection-p...
4,Halyard Health,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47463,"Size:X-Large, Brand:STERLING ZERO*, Color:Ster...",https://products.halyardhealth.com/infection-p...


In [60]:
a = upcs[['product_number', 'case_gtin', 'box_gtin']]
b = upcs2[['product', 'case_gtin', 'box_gtin']]

upc_merge = pd.merge(a, b, how='outer', left_on='product_number', right_on='product', suffixes=('_2', '_1'))
upc_merge.head()

,product_number,case_gtin_2,box_gtin_2,product,case_gtin_1,box_gtin_1
0,47459,NaN,NaN,47459,NaN,NaN
1,47460,NaN,NaN,47460,NaN,NaN
2,47461,NaN,NaN,47461,NaN,NaN
3,47462,NaN,NaN,47462,NaN,NaN
4,47463,NaN,NaN,47463,NaN,NaN


In [71]:
# upc_merge = upc_merge[['product', 'case_gtin_1', 'box_gtin_1', 'product_number', 'case_gtin_2', 'box_gtin_2']]
# upc_merge = upc_merge.rename(columns={'product':'product_1', 'product_number':'product_2'})
upc_merge

,product_1,case_gtin_1,box_gtin_1,product_2,case_gtin_2,box_gtin_2
0,47459,NaN,NaN,47459,NaN,NaN
1,47460,NaN,NaN,47460,NaN,NaN
2,47461,NaN,NaN,47461,NaN,NaN
3,47462,NaN,NaN,47462,NaN,NaN
4,47463,NaN,NaN,47463,NaN,NaN
5,14259,NaN,NaN,14259,NaN,NaN
6,44992,820170072627,820170072610,44992,820170072627,820170072610
7,44993,820170072641,820170072634,44993,820170072641,820170072634
8,44994,820170072665,820170072658,44994,820170072665,820170072658
9,44995,NaN,820170072672,44995,NaN,820170072672


In [93]:
upc_compare = upc_merge[np.logical_and((upc_merge.box_gtin_1 != 'NaN'), (upc_merge.box_gtin_2 != 'NaN'))]
upc_compare[(upc_compare.box_gtin_1 != upc_compare.box_gtin_2)]
type(upc_compare.loc[350, :].box_gtin_2) #== upc_compare.loc[350, :].box_gtin_2

float

In [108]:
# upcs2 = upcs2.rename(columns={'product':'product_number'})
# slice_ = ['product_number', 'box_gtin', 'case_gtin']
df_total2 = df_total2.merge(upcs2[slice_], how='left', on='product_number')
# upcs2[~upcs2.product_number.isin(df_total2.product_number)]

In [109]:
df_total2

,mfg,group,sub-group,product_line,product_number,description,url,box_gtin,case_gtin
0,Halyard Health,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47459,"Size:X-Small, Brand:STERLING ZERO*, Color:Ster...",https://products.halyardhealth.com/infection-p...,NaN,NaN
1,Halyard Health,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47460,"Size:Small, Brand:STERLING ZERO*, Color:Sterli...",https://products.halyardhealth.com/infection-p...,NaN,NaN
2,Halyard Health,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47461,"Size:Medium, Brand:STERLING ZERO*, Color:Sterl...",https://products.halyardhealth.com/infection-p...,NaN,NaN
3,Halyard Health,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47462,"Size:Large, Brand:STERLING ZERO*, Color:Sterli...",https://products.halyardhealth.com/infection-p...,NaN,NaN
4,Halyard Health,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47463,"Size:X-Large, Brand:STERLING ZERO*, Color:Ster...",https://products.halyardhealth.com/infection-p...,NaN,NaN
5,Halyard Health,Medical Exam Gloves,Nitrile,PURPLE NITRILE-XTRA* Exam Glove,14259,"Size:X-Small, Fit Type:Paired, Sterile:Yes, Cu...",https://products.halyardhealth.com/infection-p...,NaN,NaN
6,Halyard Health,Medical Exam Gloves,Nitrile,PURPLE NITRILE* MAX Exam Glove,44992,"Size:Small, Brand:PURPLE NITRILE*, Brand:HALYA...",https://products.halyardhealth.com/infection-p...,820170072610,820170072627
7,Halyard Health,Medical Exam Gloves,Nitrile,PURPLE NITRILE* MAX Exam Glove,44993,"Size:Medium, Brand:PURPLE NITRILE*, Brand:HALY...",https://products.halyardhealth.com/infection-p...,820170072634,820170072641
8,Halyard Health,Medical Exam Gloves,Nitrile,PURPLE NITRILE* MAX Exam Glove,44994,"Size:Large, Brand:PURPLE NITRILE*, Brand:HALYA...",https://products.halyardhealth.com/infection-p...,820170072658,820170072665
9,Halyard Health,Medical Exam Gloves,Nitrile,PURPLE NITRILE* MAX Exam Glove,44995,"Size:X-Large, Brand:PURPLE NITRILE*, Brand:HAL...",https://products.halyardhealth.com/infection-p...,820170072672,NaN


In [111]:
upcs2 = upcs2[~upcs2.product_number.isin(df_total2.product_number)]

In [112]:
upcs2['group'] = 'Medical Exam Gloves'

In [114]:
upcs2 = upcs2.rename(columns={'style':'product_line'})

In [116]:
upcs2['sub-group'] = 'Nitrile'

In [117]:
upcs2 = upcs2[df_total2.columns]

In [120]:
df_total2 = upcs2.append(df_total2)

In [123]:
df_total2 = df_total2.reset_index(drop=True)

In [128]:
df_total2.iloc[211].description = df_total2.iloc[211].description.replace('\n', '')

In [129]:
df_total2.to_csv('output/df_total_gtin.csv', index=False)